# Limpieza de los datos

In [15]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
from collections import Counter
import re
import nltk
nltk.download(['punkt','stopwords','wordnet','words'])
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
#import unidecode

[nltk_data] Downloading package punkt to /home/cmejia3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cmejia3/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/cmejia3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /home/cmejia3/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [16]:
from nltk import *
from nltk.corpus import *

### input_file_path es la ruta donde se encuentran los archivos de texto plano

In [17]:
base_path =  Path("~").expanduser().resolve()
#base_path = Path.cwd().expanduser().resolve()
input_file_path  = base_path / 'datasets/papers-txt/'
#datasetOut =base_path / "datasets/salidas_procesamiento/"
#datasetOut_freq = base_path / "datasets/salidas_freq/"

In [18]:
input_file_path

PosixPath('/home/cmejia3/datasets/papers-txt')

# Preprocesamiento

In [19]:
def fileSize(fileIn):
    size=os.stat(fileIn).st_size
    return size

def wordsCount(contenido):
    totWwords=contenido.split()
    return len(totWwords)

def cleanWordCount(contenido):
    contenido =re.sub('((f|ht)tp(s?)://)?(.*)[.][a-z]+(((\/.*(\/?)){1,}?)(.*([.].*)?))?',' ',contenido) # Eliminar las URL
    #contenido =re.sub('REFERENCES (\S|\w)+',' ',contenido) # Eliminar la bibliografia
    contenido =re.sub('[a-zA-Z0-9.?{}]+@\w+\.\w+.\w*','',contenido) # Eliminar los correos
    contenido =re.sub('\[[a-zA-Z0-9\,\. ]+\]','',contenido) # Eliminar cualquier contenido entre corchetes
    contenido =re.sub('\([a-zA-Z0-9\,\.\- ]+\)',' ',contenido) # Eliminar cualquier contenido entre paréntesis
    contenido =re.sub('((et al\.)|(i\.i\.d\.)|(i\.e\.)|\'|\’|\`)','',contenido) # Eliminar abreviaciones, apostrofes y guion
    contenido =re.sub('(á|à|ä)','a',contenido) # Reemplazar a acentuada
    contenido =re.sub('(é|è|ë)','e',contenido) # Reemplazar e acentuada
    contenido =re.sub('(í|ì|ï)','i',contenido) # Reemplazar i acentuada
    contenido =re.sub('(ó|ò|ö)','o',contenido) # Reemplazar o acentuada
    contenido =re.sub('(ú|ù|ü)','u',contenido) # Reemplazar u acentuada
    contenido =re.sub('[^a-zA-Z_|\-\s]',' ',contenido) # Eliminar caracteres que no sean: letra, número o vocales acentuadas
    contenido =re.sub(' +',' ',contenido) # Eliminar espacios en blanco
    contenido =re.sub('(a-z|A-Z){1,1}','',contenido) # Eliminar palabras o números de un caracter de longitud  
    totWwords=contenido.split()
    setWords = set(totWwords)
    #print("Total de palabras {}".format(len(totWwords)))
    #print("Total de palabras después del pre-procesamiento: {}".format(totWordDepurado))
    return len(totWwords),contenido,setWords

# Procesamiento

In [20]:
def lang_ratio(input):
    lang_ratio = {}
    tokens = wordpunct_tokenize(input)
    words = [word.lower() for word in tokens]
    for language in stopwords.fileids():
        stopwords_set = set(stopwords.words(language))
        word_set = set(words)
        common_elements = word_set.intersection(stopwords_set)
        lang_ratio[language] = len(common_elements)
    return lang_ratio

def detect_language(input):
    ratios = lang_ratio(input)
    language = max(ratios, key = ratios.get)
    return language

Conjunto de las stop words que serán eliminadas ya que no aportan valor:

In [21]:
stopWords = stopwords.words('english')

Palabras que considermos StopWords que no estan incluidas en el conjunto descargado:

In [22]:
listoStopWords = ['www','https','html','figure', 'chapter','abbcbccabcabcabcabcbcbabacba']

Se añaden las palabras que consideramos al conjunto principal

In [23]:
stopWords.extend(listoStopWords)

In [24]:
dictonary = nltk.corpus.words.words()

### Función principal procesamiento

In [25]:
def clean_files(texto,stopWords):
    
    #Quitar todos los acentos
    #texto = unidecode.unidecode(texto)
    
    #Quitar todos los caracteres especiales
    texto = re.sub('[^A-Za-z0-9]+',' ',texto)
    
    #Pasar todo a minisculas
    texto = texto.lower()
    
    #Tokenizar
    tokens = texto.split()
    
    #Variable que guarda el año en el que estamos que es el limite superior de los números que no se van a eliminar
    currentYear = int(dt.datetime.now().year)
    
    #Verificar que las palabras tengan más de un caracter, que además sean solo sean letras
    # o si son números que esten entre un rango que sea admisible para no eliminar información de año que se mencione en los artículos
    # y finalmente que no sean palabras que estan en el dicccionario de stopwords.
    
    #tokens = [w for w in tokens if (len(w)>1)&(w.isalpha() or (w.isnumeric() and int(w)>=1800 and int(w)<=currentYear))&(w not in stopWords)]
    tokens = [w for w in tokens if (len(w)>1)&(w.isalpha())&(w not in stopWords)]
    
    matcher= re.compile(r'(.)\1*')
    tokens1 = []
    for word in tokens:
        aux = [len(match.group()) for match in matcher.finditer(word)]
        if max(aux)<=3:
            tokens1.append(word)
        else:
            print(word)
    
    
    #tokens1 = [w for w in tokens1 if (Counter(w).most_common(1)[0][1]<5)]
    #eliminadas = [w for w in tokens1 if (Counter(w).most_common(1)[0][1]>=5)]
    #Stemmer
    ps = PorterStemmer() 
    tokens1 = [ps.stem(w) for w in tokens1]
    
    #Lematización
    word_net_lemmatizar = WordNetLemmatizer()

    tokens1 = [word_net_lemmatizar.lemmatize(w, pos = "v") for w in tokens1]
    
    #Se retorna el texto nuevamente en un solo string luego de ser procesado
    to_return = ' '.join(tokens1)
    
    #Se retorna el vocabulario de cada documento
    set_words = set(tokens1)
    
    #Y la frecuencia de las palabras
    freq = nltk.FreqDist(tokens1)
    return to_return,set_words,freq

Inicialización de los conjuntos:

    - Vocabulary: el conjunto de todas las palabras que contienen los documentos
    - results_text: la lista con los documentos ya organizados para construir el bag of words
    - results_frecuency: información de cada documento de las palabras que contiene cuántas veces las contiene

In [26]:
vocabulary = set()
results_text = []
results_frecuency = []

## Limpieza de los datos

In [27]:
fileSummary = "CleanSummary.csv"

contenido= "Archivo" + ";" + "Tamaño(K)" + ";" + "Cant Palabras Inicial" + ";" + "Cant Palabras depuradas"+ ";" +"Porc Limpieza"+ "\n"
resultado = pd.DataFrame()
indexFiles = []
documents = []

for f in input_file_path.glob('*.txt'):
    #Peso del archivo
    tmpSize=round(fileSize(f)/1014)
    
    #Lectura del archivo
    input_file = open(f, "r", encoding = 'utf-8')
    input_aux = input_file.read()
    
    #Cuenta de palabras iniciales
    tmpWordsOri=wordsCount(input_aux)
    #out = datasetOut / str(f).split('/')[-1]
    
    #Cuenta de palabras despues de la limpieza
    tmpWordsEnd,text,setWord=cleanWordCount(input_aux)
    #tmpPerClean=round((tmpWordsEnd/tmpWordsOri)*100)
    
    
    #Detectando el idioma
    aux = detect_language(text)
    
    if(aux == 'english'):
        text_cleanned,set_words,freq = clean_files(text,stopWords)
        #out2 = datasetOut / str(f).split('/')[-1]
        documents.append(text_cleanned)
        vocabulary = vocabulary.union(set_words)
        results_text.append(text_cleanned)
        indexFiles.append(str(f).split('/')[-1])
        
        #Escritura de los resultados del preprocesamiento
        auxRes= pd.DataFrame({'Archivo': str(f).split('/')[-1], 'Tamaño(K)': [tmpSize], 'Cant Palabras Inicial': [tmpWordsOri],'Cant Palabras depuradas': [tmpWordsEnd],"Vocabulario Inicial":len(setWord),"Vocabulario Final":len(set_words)})
        resultado = pd.concat([resultado, auxRes])
        #break
    else:
        print(aux + ':' + str(f))

resultado.to_csv(fileSummary, sep = ';', index = False)

xxxx
vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
xxxxxxx
xxxxxxx
aaaa
aaaaa
aaaaaa
baaaaa
yyyy
yyyy
xxxxxxx
yyyyyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
cacgcaccccctcg
cacgcaccccctcg
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
xxxxx
xxxxx
looooooooooooooooooooooooooooooomooooooooooooooooooooooooooooooon
looooooooooooomooooooooooooon
loooooooooooooomoooooooooooooon
aaaab
german:/home/cmejia3/datasets/papers-txt/1508.02340.txt
xxxx
yyyy
oooo
xxxxx
yyyy
yyyy
yyyy
pppperspectives
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
yyyy
xxxx
xxxx
ffff
fffff
ffff
ffff
ffff
xxxxxxxx
xxxxx
xxxx

In [28]:
resultado.head()

,Archivo,Tamaño(K),Cant Palabras Inicial,Cant Palabras depuradas,Vocabulario Inicial,Vocabulario Final
0,1410.2670.txt,24,3926,3348,923,597
0,1404.3626.txt,35,6247,4796,1052,635
0,1405.0149.txt,34,7257,5240,907,580
0,1409.2612.txt,21,4211,2960,506,305
0,1511.00867.txt,72,14471,12117,1418,862


In [29]:
len(vocabulary)

70148

## Construcción del Bag of Words

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import vstack,save_npz,load_npz

Construido el vocabulario podemos construir el bag of words, que se hace con la ayuda de la funcion CountVectorizer

In [31]:
vectorizer = CountVectorizer(analyzer = "word",vocabulary =vocabulary , tokenizer = None, preprocessor = None, stop_words = 'english', max_features = 5000) 
train_data_features = vectorizer.fit_transform(documents)

In [32]:
len(vocabulary)

70148

In [33]:
save_npz('sparse_matrix.npz', train_data_features)
#sparse_matrix = load_npz('sparse_matrix.npz')

In [34]:
vectorizer.vocabulary_

{'aa': 0,
 'aaa': 1,
 'aaab': 2,
 'aaacahichvhlssnafd': 3,
 'aaacanichvhlssnafd': 4,
 'aaacfxichvhlsinbfd': 5,
 'aaacpnichvhlbhmxfd': 6,
 'aaaczhichvhlssnafd': 7,
 'aaai': 8,
 'aaaibarri': 9,
 'aab': 10,
 'aaba': 11,
 'aabaabc': 12,
 'aabab': 13,
 'aababa': 14,
 'aababaababb': 15,
 'aabb': 16,
 'aabbb': 17,
 'aabc': 18,
 'aabcdaccaac': 19,
 'aachen': 20,
 'aad': 21,
 'aaecc': 22,
 'aaem': 23,
 'aag': 24,
 'aagaard': 25,
 'aagea': 26,
 'aai': 27,
 'aaihaa': 28,
 'aain': 29,
 'aaini': 30,
 'aaj': 31,
 'aakv': 32,
 'aalbersberg': 33,
 'aalborg': 34,
 'aalg': 35,
 'aall': 36,
 'aalto': 37,
 'aam': 38,
 'aama': 39,
 'aamodt': 40,
 'aan': 41,
 'aana': 42,
 'aand': 43,
 'aanderaa': 44,
 'aanstad': 45,
 'aanund': 46,
 'aarabi': 47,
 'aardal': 48,
 'aarhu': 49,
 'aaron': 50,
 'aaronson': 51,
 'aart': 52,
 'aarti': 53,
 'aas': 54,
 'aashtiani': 55,
 'aat': 56,
 'aau': 57,
 'aaw': 58,
 'aawcondit': 59,
 'aazhang': 60,
 'ab': 61,
 'aba': 62,
 'abab': 63,
 'ababa': 64,
 'ababac': 65,
 'ababb': 66,


In [35]:
with open('vocabulary.txt', 'w') as f:
    for item in vectorizer.get_feature_names():
        f.write("%s\n" % item)

In [36]:
import pickle
filename = 'BoW1.sav'
pickle.dump(vectorizer, open(filename, 'wb'))
#loaded_model = pickle.load(open('BoW1.sav', 'rb'))

In [37]:
#words = set(nltk.corpus.words.words())

In [38]:
#len(vocabulary)

In [39]:
#print(vocabulary - words)